# Assignment 2

In [ ]:
#Import Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from scipy.stats import norm
from math import pi
%matplotlib inline

### Load Dataset

In [ ]:
#Import Data
stock_data = pd.read_excel('PS1-2_Studynet.xlsx')

#Clean Header
stock_data.columns = stock_data.columns.str.replace("Equity", "")
stock_data.columns = stock_data.columns.str.replace("SE", "")
stock_data.columns = stock_data.columns.str.replace("SW", "")
stock_data.columns = stock_data.columns.str.replace(" ", "")
stock_data = stock_data.set_index('Date')
stock_data = stock_data.loc['2010-01-01':'2017-12-29']

stock_data['LISN'] = stock_data.LISN / stock_data.LISN.shift(1) - 1
stock_data['BAER'] = stock_data.BAER / stock_data.BAER.shift(1) - 1
stock_data.head()

#print(type(stock_data.index))

### Excercise 1.1

In [ ]:
def emp_mad(data):
    return np.mean(np.absolute(data - np.mean(data)))

def norm_mad(data):
    return np.sqrt(2/pi)*np.std(data)

def semidev(data):
    return np.sqrt(np.mean(np.power(np.minimum(data,np.mean(data)),2)))

def exp_shortfall(alpha,data):
    #condiational variance, expected shortfall
    
    mu = np.mean(data)
    sigma = np.std(data)
    
    return (1 - alpha)** -1 * norm.pdf(norm.ppf(alpha))*sigma - mu

### Excersise 1.2

In [ ]:
def emp_expshort(data, alpha):
    
    def delta(ret, vara):
        if ret < (vara*-1):
            return 1
        else:
            return 0
    
    def varalpha(data, alpha):
        return - (np.mean(data) - norm.ppf(alpha)*np.std(data))
    
    vara = varalpha(data, alpha)
    
    sum_delta = []
    
    for i in range(0,len(data)):
        sum_delta.append(delta(data[i], vara))

    ES = (-1/np.sum(sum_delta))*np.sum(sum_delta * data)
    
    return ES 

print(emp_expshort(stock_data['LISN'], 0.01))
print(emp_expshort(stock_data['LISN'], 0.05))

In [ ]:
output = {'LISN':   [round(np.std(stock_data['LISN']),6), 
                         emp_mad(stock_data['LISN']),
                         norm_mad(stock_data['LISN']),
                         semidev(stock_data['LISN']), 
                         exp_shortfall(0.05,stock_data['LISN']), 
                         exp_shortfall(0.01,stock_data['LISN']), 
                         emp_expshort(stock_data['LISN'], 0.05),
                         emp_expshort(stock_data['LISN'], 0.01)], 
          
          'BAER':   [round(np.std(stock_data['BAER']),6), 
                         emp_mad(stock_data['BAER']),
                         norm_mad(stock_data['BAER']),
                         semidev(stock_data['BAER']), 
                         exp_shortfall(0.05,stock_data['BAER']), 
                         exp_shortfall(0.01,stock_data['BAER']),
                         emp_expshort(stock_data['BAER'], 0.05),
                         emp_expshort(stock_data['BAER'], 0.01)]}

ouput_table = pd.DataFrame(data = output, index = ['Std', 'EmpMad','NormMad','Semi Dev', 
                                                   'Exp. Shortfall 95%',
                                                   'Exp. Shortfall 99%', 
                                                   'Empirical ES 95%',
                                                   'Empirical ES 99%',])
ouput_table

### Excersice 1.3

In [ ]:
initial_amount = 10000000

def invested_amount_std(data, initial_amount, period = 4):
    
    value_position = [initial_amount]
    
    for i in range(1,len(data)):
        
        value_position.append((1 + data[i]) * value_position[i-1])
        
    return np.round(np.std(value_position[0:4]),4)
    
print(invested_amount_std(stock_data['LISN'], initial_amount))
print(invested_amount_std(stock_data['BAER'], initial_amount))

### Excersise 1.4

In [ ]:
returns_sorted = stock_data.sort_values('LISN')['LISN']
sdt_norm = pd.DataFrame(np.random.normal(np.mean(stock_data['LISN']),np.std(stock_data['LISN']),size = len(stock_data['LISN'])))
sdt_norm = sdt_norm.sort_values(0)


plt.plot(sdt_norm, returns_sorted, color = 'grey', label = 'QQ', marker ='o')

plt.xlabel('Normal Distribution')
plt.ylabel('Empirical Returns')

plt.show()

In [ ]:
plt.hist(stock_data['LISN'][1:], 50 ,density = True, facecolor='g', alpha=0.75)
p = np.random.normal(np.mean(stock_data['LISN']),np.std(stock_data['LISN']),size = len(stock_data['LISN']))

plt.show()

### Excersise 2

In [ ]:
stock_data2 = pd.read_excel('PS2-2.xlsx')
stock_data2 = stock_data2.set_index('Date')

cols = ['WMT (Walmart)', 
        'AAPL (Apple)', 
        'DIS (The Walt Disney Company)', 
        'IBM (International Business Machines Corporation)', 
        'JNJ (Johnson & Johnson)',
        'KO (The Coca-Cola Company)',
        'NKE (NIKE, Inc.)',
        'TXN (Texas Instruments Incorporated)']

for i in range(0,len(cols)):
    stock_data2['Return ' + cols[i]] = stock_data2[cols[i]]/stock_data2[cols[i]].shift(1) - 1

cols = stock_data2.columns

#Drop fist row with Nan Returns
stock_data2 = stock_data2.drop(stock_data2.index[0])
#stock_data2[stock_data2.name != 'nan']

stock_data2.head()

### Excersise 2.1

In [ ]:
def varalpha(data, alpha):
    return - (np.mean(data) - norm.ppf(alpha) * np.std(data))

In [ ]:
output2 = {cols[8]:   [varalpha(stock_data2[cols[8]], 0.05),  varalpha(stock_data2[cols[8]], 0.01),  emp_expshort(stock_data2[cols[8]], 0.05),  emp_expshort(stock_data2[cols[8]], 0.01)],  
           cols[9]:   [varalpha(stock_data2[cols[9]], 0.05),  varalpha(stock_data2[cols[9]], 0.01),  emp_expshort(stock_data2[cols[9]], 0.05),  emp_expshort(stock_data2[cols[9]], 0.01)], 
           cols[10]:  [varalpha(stock_data2[cols[10]], 0.05), varalpha(stock_data2[cols[10]], 0.01), emp_expshort(stock_data2[cols[10]], 0.05), emp_expshort(stock_data2[cols[10]], 0.01)], 
           cols[11]:  [varalpha(stock_data2[cols[11]], 0.05), varalpha(stock_data2[cols[11]], 0.01), emp_expshort(stock_data2[cols[11]], 0.05), emp_expshort(stock_data2[cols[11]], 0.01)], 
           cols[12]:  [varalpha(stock_data2[cols[12]], 0.05), varalpha(stock_data2[cols[12]], 0.01), emp_expshort(stock_data2[cols[12]], 0.05), emp_expshort(stock_data2[cols[12]], 0.01)], 
           cols[13]:  [varalpha(stock_data2[cols[13]], 0.05), varalpha(stock_data2[cols[13]], 0.01), emp_expshort(stock_data2[cols[13]], 0.05), emp_expshort(stock_data2[cols[13]], 0.01)], 
           cols[14]:  [varalpha(stock_data2[cols[14]], 0.05), varalpha(stock_data2[cols[14]], 0.01), emp_expshort(stock_data2[cols[14]], 0.05), emp_expshort(stock_data2[cols[14]], 0.01)],
           cols[15]:  [varalpha(stock_data2[cols[15]], 0.05), varalpha(stock_data2[cols[15]], 0.01), emp_expshort(stock_data2[cols[15]], 0.05), emp_expshort(stock_data2[cols[15]], 0.01)], 
          }

ouput_table2 = pd.DataFrame(data = output2, index = ['95% Var', '99% Var', 'Emp Var 95%', 'Emp Var 99%'])
ouput_table2

### Excersise 2.2

In [ ]:
xls = pd.ExcelFile('PS2-2.xlsx')
apple_data = pd.read_excel(xls, 'AAPL')
apple_data = apple_data.set_index('Date')
apple_data['Return AAPL (Apple)'] = apple_data['AAPL Adj Close']/apple_data['AAPL Adj Close'].shift(1) - 1

apple_returns = list(stock_data2['Return AAPL (Apple)'])
apple_data2 = list(apple_data['Return AAPL (Apple)'])
apple_data2[0] = 0.010326

apple_series = apple_returns + apple_data2

In [ ]:
#print(varalpha(stock_data2[cols[9]], 0.05))
rolling_var = []
for i in range(0,20):
    
    rolling_var.append(varalpha(apple_series[0+i:756+i], 0.05))

var_exceeded = [0]*20
for i in range(0,19):
    
    if rolling_var[i] > apple_series[756+i]:
        var_exceeded[i] = 1

print(var_exceeded)        

In [ ]:
print(np.round(rolling_var,4))

In [ ]:
print(np.round(apple_data[756:],4))

### Excersise 2.3

In [ ]:
def portfolio_ret(w, expected_returns):
    return np.round(np.sum(expected_returns * w),6)

def portfolio_vol(w, cov_mat):
    return np.dot(np.dot(w.T,cov_mat), w)

In [ ]:
portfolio_weights = np.divide([55000, 15000, 23400, 43200, 131000, 321000, 104000, 38700],
                              np.sum([55000, 15000, 23400, 43200, 131000, 321000, 104000, 38700]))

assets_mean = np.mean(stock_data2[cols[8:]])

cov_mat = np.round(np.cov(stock_data2.iloc[:,8:].values.T[:]),6)

portfolio_return = portfolio_ret(portfolio_weights, assets_mean)

portfolio_std = np.sqrt(portfolio_vol(portfolio_weights, cov_mat))

print(portfolio_return)
print(portfolio_std)

In [ ]:
portfolio_value_at_risk =  - portfolio_return + norm.ppf(1-0.05)*portfolio_std
print(portfolio_value_at_risk)

### Excercise 2.4

In [ ]:
def portfolio_return_series(assets, weights):
    #assets is return series 
    p_retseries = []
    
    for i in range(0,len(assets.iloc[:,8:].values)):
        p_retseries.append(np.sum((assets.iloc[:,8:].values[i] * portfolio_weights)[0]))
    
    return p_retseries

p_retseries = portfolio_return_series(stock_data2, portfolio_weights)

In [ ]:
def marginal_port_val(alpha, data):
    
    z = norm.ppf(1-alpha)
    
    sigma_ip = np.cov(data, p_retseries)[0][1]
    
    sigma_P = portfolio_std
    
    marginal_var = (sigma_ip/sigma_P) * z
    
    return marginal_var


In [ ]:
for i in range(8,15):
    
    print(cols[i])
    print(marginal_port_val(0.05, stock_data2[cols[i]]))
    
    marginal_var = []
    marginal_var.append(marginal_port_val(0.05, stock_data2[cols[i]]))

###  Excercise 2.5

In [ ]:
print(portfolio_weights*marginal_var/portfolio_value_at_risk)

###  Excercise 2.6

In [ ]:
#See word

###  Excercise 2.7

In [ ]:
np.random.seed(1)

time_series = np.zeros((1000,len(stock_data2['Return AAPL (Apple)'])))

for i in range(0,999):
    randint = np.int_(np.random.uniform(low = 1, high = len(stock_data2['Return AAPL (Apple)']), size = 1000))
    
    for j in range(1,len(stock_data2['Return AAPL (Apple)'])):
        time_series[i][j] = stock_data2['Return AAPL (Apple)'][randint[j]]  

In [ ]:
st_dev = []
emp_var99 = [] 
emp_var95 = []


for i in range(0,999):
    st_dev.append(np.std(time_series[i]))
    emp_var99.append(varalpha(time_series[i], 0.01))
    emp_var95.append(varalpha(time_series[i], 0.05))
    
std_sorted = np.sort(st_dev)
emp_var99_sorted = np.sort(emp_var99)
emp_var95_sorted = np.sort(emp_var95)

In [ ]:
plt.hist(std_sorted, bins = np.linspace(std_sorted.min(),std_sorted.max()), color = 'red')
plt.show()

plt.hist(emp_var99_sorted, bins = np.linspace(emp_var99_sorted.min(),emp_var99_sorted.max()), color = 'blue')
plt.show()

plt.hist(emp_var95_sorted, bins = np.linspace(emp_var95_sorted.min(),emp_var95_sorted.max()), color = 'grey')
plt.show()